<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/Rest_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
def add(lo_left,lo_right,la_left,la_right):
    a = 0.008 #调整经纬度改这里
    lo_left -= a
    lo_right += a
    la_left -= a
    la_right += a
    return lo_left, lo_right, la_left, la_right

In [3]:
def judgeType(item):
  if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):#改type在这里
    return True

In [4]:
def calSqr(sqr):
    if sqr <= 1100:
        left = sqr - 100
        right = sqr + 100
    elif sqr <= 2200:
        left = sqr - 200
        right = sqr + 200
    else:
        left = sqr - 300
        right = sqr + 300
    return left, right

In [5]:
def calYear(year):
  if year < 1990:
     begin = 0 
     end = 1989
  elif year >= 1990 and year < 2011:
    begin = 1990
    end = 2010
  elif year >= 2011 and year < 2022:
    begin = 2011
    end = 2021
  else:
    begin = 2022
    end = 9999
  return begin,end

In [6]:
def FindifNull(research, lo_left,lo_right,la_left,la_right, status, date, sq_left, sq_right,begin_year,end_year):
  #经纬度查找
  filter = {
        "Longitude": "{}-{}".format(lo_left, lo_right),
        "Latitude": "{}-{}".format(la_left, la_right),
        "MlsStatus" : "{}".format(status),
        "CloseDate" : "{}".format(date),
        "LivingArea" : "{}-{}".format(sq_left, sq_right),
        "YearBuilt" :"{}-{}".format(begin_year, end_year)
        }
  result = research.search(resource='Property', resource_class='Property', search_filter=filter)
  lis = []
  count = 0
  di ={}
  df = pd.DataFrame()
  for item in result:
    if(judgeType(item)):
      df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
      count += 1
      di = item # 存表头
  for key in di:
    lis.append(key)
  if(count >= 5): 
    df = df.T
    df.columns = lis
    return df
  else:
    lo_left, lo_right, la_left, la_right = add(lo_left,lo_right,la_left,la_right)
    return FindifNull(research, lo_left,lo_right,la_left,la_right, status, date, sq_left, sq_right,begin_year,end_year)

In [7]:
def FindHouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,zip,column, sub, status, date, sq_left, sq_right,begin_year, end_year):
  #sub，zip查找，未找到足够的自动转经纬度
  filter = {
        "PostalCode" : zip,
        "MlsStatus" : "{}".format(status),
        "CloseDate" : "{}".format(date),
        "LivingArea" : "{}-{}".format(sq_left, sq_right),
        "YearBuilt" :"{}-{}".format(begin_year, end_year)
        }
  result = research.search(resource='Property', resource_class='Property', search_filter=filter)
  count = 0
  df = pd.DataFrame()
  for item in result: 
    if(judgeType(item)):
      if(sub == item['SubdivisionName'].split()[0]):
         df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
         count += 1    
  if(count >= 5):  
    df = df.T
    df.columns = column # 
    return df
  else:
    return FindifNull(research, house_leftLo,house_rightLo, house_leftLa, house_rightLa, status, date, sq_left, sq_right,begin_year, end_year)

In [8]:
def findfirst(result):
  #判断是否有，不用管
    count = 0
    for item in result:
        count += 1
    if(count == 0):
        return False
    else:
        return True    

In [33]:
def insertId(pulled_value, mls,rets_client, column2, status, date):
  #mls查找，按照先sub + zip，后geo
  column = []
  for key in pulled_value:
    column.append(key)
  print("\nThis value exists in Website")
  zip = pulled_value.get('PostalCode')
  latitude = pulled_value.get('Latitude')
  longitude = pulled_value.get('Longitude')
  bedroom = pulled_value.get('BedroomsTotal')
  sqr = pulled_value.get('LivingArea')
  year = pulled_value.get('YearBuilt')
  self_price = float(pulled_value.get('ClosePrice')) #####
  if(zip != "" and latitude != "" and longitude != "" and sqr != "" and year != ""):
    house_leftLa = float(pulled_value.get('Latitude'))
    house_rightLa = float(pulled_value.get('Latitude')) 
    house_leftLo = float(pulled_value.get('Longitude'))
    house_rightLo = float(pulled_value.get('Longitude'))
    sub = str(pulled_value.get('SubdivisionName').split()[0])
    sqr = float(sqr)
    year = int(year)
    left, right = calSqr(sqr)
    begin, end = calYear(year)
    result1 = FindHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, rets_client,int(zip) ,column, sub, status, date, left, right,begin,end) 
    result1['FullAddress'] = result1['StreetNumber'] + " " + result1['StreetName']  + ", " + result1['City'] + ", " +result1['StateOrProvince'] + " " + result1['PostalCode']       
    result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
  'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
  'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
  'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
  'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
  'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
  'USProperty_MUI', 'YearBuilt','LotSizeAcres','FullAddress']]
    price = []
    cdom_signValue = []
    df_id = pd.DataFrame()

###########################################################
# compares
      
    if(status == 'SLD'): 
      for item in result1['ClosePrice']:
        item = float(item)
        price.append(item)
      result1['ClosePrice'] = price
      mean1 = result1['ClosePrice'].mean()
      top1 = list(result1['ClosePrice'])
      top1.sort(reverse = True)
      top1_mean = sum(top1[0:3])/3
      min1 = list(result1['ClosePrice'])
      min1.sort(reverse = False)
      median = result1['ClosePrice'].median()
      percent_mean1 = ((mean1 - self_price)/ self_price)*100 ######
      percent_Top_mean1 = ((top1_mean - self_price)/self_price)*100 ######
      percent_Top1 = ((top1_mean - self_price)/self_price)*100 ######
      for item in result1['CumulativeDaysOnMarket']:
        item = int(item)
        cdom_signValue.append(item)
      result1['CumulativeDaysOnMarket'] = cdom_signValue
      cdom = result1['CumulativeDaysOnMarket'].mean()
      df_id_comp = pd.DataFrame()
      df2_append_comp = {'Min': min1[0],'Median': median ,'Mean': mean1,'Top 3 Mean': top1_mean,'Max': top1[0] ,'CDOM': cdom}
      df_id_comp = df_id_comp.append(df2_append_comp, ignore_index = True)
      df3_append_comp = {'Min': None,'Median': None ,'Mean': percent_mean1,'Top 3 Mean': percent_Top_mean1,'Max': percent_Top1 ,'CDOM': None} ######
      df_id_comp = df_id_comp.append(df3_append_comp, ignore_index = True)####
      df4_append_comp = {'Min': None,'Median': None ,'Mean': 10,'Top 3 Mean': 20,'Max':30 ,'CDOM': None} ######
      df_id_comp = df_id_comp.append(df4_append_comp, ignore_index = True)#####
      df_id_comp = df_id_comp.rename(index={0: 'Comps', 1: '% above price',2: 'Est. Margin'})#####
      df_id_comp.to_csv('{}.csv'.format(mls))
###########################################################
    df_id_status = pd.DataFrame()		
    for i in range(len(result1['FullAddress'])):								
      df_id_status = df_id_status.append(result1.loc[i][column2], ignore_index = True)
    return df_id_status
  else:
    print('Information missing in the system!')

In [10]:
def insertAdd(number, name, city, state, zip, address, rets_client, column2, status, date):
  #add查找，按照先sub + zip，后geo
  pulled_value = {}
  result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(StreetNumber = {}),(StreetName = {}),(PostalCode = {})'.format(number,name,zip))
  for item in result:
    pulled_value = item
  if(len(pulled_value) != 0):
    column = []
    for key in pulled_value:
      column.append(key)
    print("\nThis value exists in Website")
    zip = pulled_value.get('PostalCode')
    latitude = pulled_value.get('Latitude')
    longitude = pulled_value.get('Longitude')
    sqr = pulled_value.get('LivingArea')
    year = pulled_value.get('YearBuilt')
    if(zip != "" and latitude != "" and longitude != "" and sqr != ""):
      house_leftLa = float(pulled_value.get('Latitude'))
      house_rightLa = float(pulled_value.get('Latitude')) 
      house_leftLo = float(pulled_value.get('Longitude'))
      house_rightLo = float(pulled_value.get('Longitude'))
      zip = int(pulled_value.get('PostalCode'))
      sqr = float(sqr)
      year = int(year)
      left, right = calSqr(sqr)
      begin, end = calYear(year)
      sub = str(pulled_value.get('SubdivisionName').split()[0])
      result1 = FindHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, rets_client,zip,column, sub, status, date, left, right,begin,end)        
      result1['FullAddress'] = result1['StreetNumber'] + " " + result1['StreetName']  + ", " + result1['City'] + ", " +result1['StateOrProvince'] + " " + result1['PostalCode']       
      result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
  'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
  'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
  'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
  'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
  'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
  'USProperty_MUI', 'YearBuilt','LotSizeAcres','FullAddress']]
      price = []
      cdom_signValue = []
      df_id = pd.DataFrame()

###########################################################
# compares
    
      if(status == 'SLD'): 
        for item in result1['ClosePrice']:
          item = float(item)
          price.append(item)
        result1['ClosePrice'] = price
        mean1 = result1['ClosePrice'].mean()
        top1 = list(result1['ClosePrice'])
        top1.sort(reverse = True)
        top1_mean = sum(top1[0:3])/3
        min1 = list(result1['ClosePrice'])
        min1.sort(reverse = False)
        percent_mean1 = (mean1 - 90) /100 ######
        percent_Top_mean1 = (top1_mean - 90) / 100 ######
        percent_Top1 = (top1_mean-90)/100 ######
        median = result1['ClosePrice'].median()
        for item in result1['CumulativeDaysOnMarket']:
          item = int(item)
          cdom_signValue.append(item)
        result1['CumulativeDaysOnMarket'] = cdom_signValue
        cdom = result1['CumulativeDaysOnMarket'].mean()
        df_id_comp = pd.DataFrame()
        df2_append_comp = {'Min': min1[0],'Median': median ,'Mean': mean1,'Top 3 Mean': top1_mean,'Max': top1[0] ,'Avg. CDOM': cdom}
        df_id_comp = df_id_comp.append(df2_append_comp, ignore_index = True)
        df3_append_comp = {'Min': None,'Median': None ,'Mean': percent_mean1,'Top 3 Mean': percent_Top_mean1,'Max': percent_Top1 ,'CDOM': None} ######
        df_id_comp = df_id_comp.append(df3_append_comp, ignore_index = True)####
        df4_append_comp = {'Min': None,'Median': None ,'Mean': 10,'Top 3 Mean': 20,'Max':30 ,'CDOM': None} ######
        df_id_comp = df_id_comp.append(df4_append_comp, ignore_index = True)#####
        df_id_comp = df_id_comp.rename(index={0: 'Comps', 1: '% above price',2: 'Est. Margin'})#####
        df_id_comp.to_csv('{}.csv'.format(address))
###########################################################
      df_id_status = pd.DataFrame()		
      for i in range(len(result1['FullAddress'])):								
        df_id_status = df_id_status.append(result1.loc[i][column2], ignore_index = True)
      return df_id_status
    else:
      print('check coordinates')
      return FindAdd(rets_client, address, city, state, zip, status, date, column2)
  else:
    print('check coordinates')
    return FindAdd(rets_client, address, city, state, zip, status, date, column2)

In [11]:
def FindAdd(rets_client, address, city, state, zipcode, bedroom, status, date, column2):  
  #geo代码，在mls和address中引用
  result = cg.address(address, city = city, state= state,zipcode = zipcode)
  lo_left = result[0]['coordinates']['x']
  lo_right = result[0]['coordinates']['x']
  la_left = result[0]['coordinates']['y']
  la_right = result[0]['coordinates']['y']
  left = 0
  right = 9999
  result = FindifNull(rets_client, lo_left, lo_right, la_left, la_right, status, date, left, right)
  result['FullAddress'] = result['StreetNumber'] + " " + result['StreetName']  + ", " + result['City'] + ", " +result['StateOrProvince'] + " " + result['PostalCode']         
  result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','LotSizeAcres','FullAddress']]

  price = []
  cdom_signValue = []
  df_id = pd.DataFrame()

###########################################################
# compares
    
  if(status == 'SLD'): 
    for item in result['ClosePrice']:
      item = float(item)
      price.append(item)
    result['ClosePrice'] = price
    mean1 = result['ClosePrice'].mean()
    top1 = list(result['ClosePrice'])
    top1.sort(reverse = True)
    top1_mean = sum(top1[0:3])/3
    min1 = list(result['ClosePrice'])
    min1.sort(reverse = False)
    median = result['ClosePrice'].median()
    percent_mean = (mean1-90)/100 ######
    percent_Top_mean = (top1_mean-90)/100######
    percent_Top = (top1_mean-90)/100 ######
    for item in result['CumulativeDaysOnMarket']:
      item = int(item)
      cdom_signValue.append(item)
    result['CumulativeDaysOnMarket'] = cdom_signValue
    cdom = result['CumulativeDaysOnMarket'].mean()
    df_id_comp = pd.DataFrame()
    df2_append_comp = {'Min': min1[0],'Median': median ,'Mean': mean1,'Top 3 Mean': top1_mean,'Max': top1[0] ,'CDOM': cdom}
    df_id_comp = df_id_comp.append(df2_append_comp, ignore_index = True)
    df3_append_comp = {'Min': None,'Median': None ,'Mean': percent_mean,'Top 3 Mean': percent_Top_mean,'Max': percent_Top ,'CDOM': None} ######
    df_id_comp = df_id_comp.append(df3_append_comp, ignore_index = True)####
    df4_append_comp = {'Min': None,'Median': None ,'Mean': 10,'Top 3 Mean': 20,'Max':30 ,'CDOM': None} ######
    df_id_comp = df_id_comp.append(df4_append_comp, ignore_index = True)#####
    df_id_comp = df_id_comp.rename(index={0: 'Comps', 1: '% above price',2: 'Est. Margin'})#####
    df_id_comp.to_csv('{} first.csv'.format(address))
###########################################################
  df_id_status = pd.DataFrame()		
  for i in range(len(result['FullAddress'])):								
    df_id_status = df_id_status.append(result.loc[i][column2], ignore_index = True)
  return df_id_status

In [12]:
def judge(i):
  #不用管，为了美观
    if i % 10 == 1:
        return 'st'
    elif i % 10 == 2:
        return 'nd'
    elif i % 10 == 3:
        return 'rd'
    else:
        return 'th'

In [25]:
def main():
  login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
  username = '0671181_NID'
  password = 'Rt$tg6jx'
  rets_client = Session(login_url, username, password)
  rets_client.login()
  while True:
        column3 = ['MlsStatus','FullAddress', 'ListingId', 'ListPrice', 'LivingArea', 'BedroomsTotal', 'BathroomsTotalInteger', 'YearBuilt', 'GarageSpaces', 'PoolYN', 'LotSizeAcres','CloseDate','CumulativeDaysOnMarket','Longitude','Latitude']
        column2 = ['MlsStatus','FullAddress', 'ListingId', 'ClosePrice', 'LivingArea', 'BedroomsTotal', 'BathroomsTotalInteger', 'YearBuilt', 'GarageSpaces', 'PoolYN', 'LotSizeAcres','CloseDate','CumulativeDaysOnMarket','Longitude','Latitude']
        print('please insert values, csv_address or csv_mls: ')
        choice = input()
        if choice == 'csv_address':
            while True:
                print('select which csv you want to use')
                csv = input()
                try:
                    data = pd.read_csv('{}.csv'.format(csv))        
                    code = []
                    name = []
                    number = []
                    street = []
                    city = []
                    state = []
                    for item in data['address']:
                        a = item.split(',')[0]
                        b = item.split(',')[2]
                        c = item.split(',')[1]
                        street.append(a)
                        city.append(c)
                        numbers = a.split()[0]
                        names = a.split()[1:(len(a.split()) - 1)]
                        codes = b.split()[1][0:5]
                        states = b.split()[0]
                        state.append(states)
                        code.append(codes)
                        name.append(names)
                        number.append(numbers)
                    finalN = []
                    for item in name:
                        string = ""
                        for items in item:
                            string += items
                            string += ' '
                        finalN.append(string[0:len(string) - 1])            
                    for i in range(len(code)):
                        df1 = insertAdd(number[i], finalN[i], city[i], state[i], code[i], street[i], rets_client, column2, "SLD", "2022-01-01-2022-06-30")
                        df2 = insertAdd(number[i], finalN[i], city[i], state[i], code[i], street[i], rets_client, column3, "PND", ".EMPTY.")
                        df3 = insertAdd(number[i], finalN[i], city[i], state[i], code[i], street[i], rets_client, column3, "ACT", ".EMPTY.")
                        df1.to_csv('{} SLD.csv'.format(finalN[i]))
                        #############
                        df2 = df2[df2['Longitude'].between(min(df1['Longitude']),max(df1['Longitude']))]
                        df2 = df2[df2['Latitude'].between(min(df1['Latitude']),max(df1['Latitude']))]
                        #############
                        df3 = df3[df3['Longitude'].between(min(df1['Longitude']),max(df1['Longitude']))]
                        df3 = df3[df3['Latitude'].between(min(df1['Latitude']),max(df1['Latitude']))]
                        frames = [df2, df3]
                        df4 = pd.concat(frames, ignore_index=True)
                        df4.to_csv('{} PND,ACT.csv'.format(finalN[i]))                        
                    return "finished"
                except FileNotFoundError:
                    print("Oops! There is no file!  Try again...")

        elif(choice =='csv_mls'):
          while True:
            print("Which csv do you want to use? (make sure you have listingId)")
            csv = input()
            try:
              lis_mls = []
              data = pd.read_csv('{}.csv'.format(csv)) 
              for item in data['mls']:
                lis_mls.append(int(item))
              for i in range(len(lis_mls)):
                pulled_value = {}
                result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(ListingId = {})'.format(lis_mls[i]))
                for items in result:
                  pulled_value = items
                if(len(pulled_value) != 0):
                  try:
                    df1 = insertId(pulled_value, lis_mls[i], rets_client, column2, "SLD", "2022-01-01-2022-06-30") 
                    df2 = insertId(pulled_value, lis_mls[i], rets_client, column3, "PND", ".EMPTY.")
                    df3 =  insertId(pulled_value, lis_mls[i], rets_client, column3, "ACT", ".EMPTY.")
                    df1.to_csv('{} SLD.csv'.format(lis_mls[i]))
                    #############
                    df2 = df2[df2['Longitude'].between(min(df1['Longitude']),max(df1['Longitude']))]
                    df2 = df2[df2['Latitude'].between(min(df1['Latitude']),max(df1['Latitude']))]
                    #############
                    df3 = df3[df3['Longitude'].between(min(df1['Longitude']),max(df1['Longitude']))]
                    df3 = df3[df3['Latitude'].between(min(df1['Latitude']),max(df1['Latitude']))]
                    frames = [df2, df3]
                    df4 = pd.concat(frames, ignore_index=True)
                    df4.to_csv('{} PND,ACT.csv'.format(lis_mls[i])) 
                    if i == len(lis_mls) - 1:
                        return
                  except ValueError:
                    print('{} is not valid'.format(lis_mls[i]))
                 #end here
            except FileNotFoundError:
              print("Oops! There is no file!  Try again...")

        elif(choice == 'value'):
            while True:
                print('How many mls you want to check?')
                number = input()
                try:
                    Id_list = []
                    for i in range(int(number)):
                        print('Enter the {}{} #MLS'.format(i+1, judge(i+1)))
                        mls = input()
                        Id_list.append(mls)
           #14730621
                    for i in range(len(Id_list)):
                        pulled_value = {}
                        result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(ListingId = {})'.format(Id_list[i]))
                        for items in result:
                            pulled_value = items
                        if(len(pulled_value) != 0):
                          df1 = insertId(pulled_value, Id_list[i], rets_client, column2, "SLD", "2022-01-01-2022-06-30") 
                          df2 = insertId(pulled_value, Id_list[i], rets_client, column3, "PND", ".EMPTY.")
                          df3 =  insertId(pulled_value, Id_list[i], rets_client, column3, "ACT", ".EMPTY.")
                          df1.to_csv('{} SLD.csv'.format(Id_list[i]))
                          #############
                          df2 = df2[df2['Longitude'].between(min(df1['Longitude']),max(df1['Longitude']))]
                          df2 = df2[df2['Latitude'].between(min(df1['Latitude']),max(df1['Latitude']))]
                          #############
                          df3 = df3[df3['Longitude'].between(min(df1['Longitude']),max(df1['Longitude']))]
                          df3 = df3[df3['Latitude'].between(min(df1['Latitude']),max(df1['Latitude']))]
                          frames = [df2, df3]
                          df4 = pd.concat(frames, ignore_index=True)
                          df4.to_csv('{} PND,ACT.csv'.format(Id_list[i])) 
                          if(i == len(Id_list) - 1):
                              return
                        else:

                            print('Id {} not exists, please insert specific value'.format(Id_list[i]))
                            print("If you don't know the address, please insert skip")
                            deci = input()
                            if(deci == 'skip'):
                                if(i == int(number) - 1):
                                    return
                                continue
                                
                            else:
                                print('Enter street number')
                                number = input()
        #'8615 Barclay Street'
                                print('Enter street name')
                                name = input()
                                print('Enter city')
                                city = input()
        #'Dallas'
                                print('Enter state')
                                state = input()
        #'Texas'
                                print('Enter zipcode')
                                zip = input()
        #'75227'
                                string = ""
                                for i in range(len(name.split()) - 1):
                                  string += name.split()[i] + " "
                                string = string[0:len(string) - 1]
                                address = number + ' ' + name
                                try:

                                  df1 = insertAdd(number, string, city, state, zip, address, rets_client, column2, "SLD", "2022-01-01-2022-06-30")
                                  df2 = insertAdd(number, string, city, state, zip, address, rets_client, column3, "PND", ".EMPTY.")
                                  df3 =  insertAdd(number, string, city, state, zip, address, rets_client, column3, "ACT", ".EMPTY.")
                                  df1.to_csv('{} SLD.csv'.format(Id_list[i]))
                                  df3 = df3[df3['Longitude'].between(min(df1['Longitude']),max(df1['Longitude']))]
                                  frames = [df2, df3]
                                  df4 = pd.concat(frames, ignore_index=True)
                                  df4.to_csv('{} PND,ACT.csv'.format(address)) 
                                  if(i == int(number) - 1):
                                      return
                                except IndexError:
                                    print("invalid input, please next value!")   

                          
                except ValueError:
                    print("invalid input, try again!")
        else:
            print("invalid input, try again!")

In [32]:
if __name__ == "__main__":
    print(main()) 
    #5005 Bama Drive, Arlington, Texas 76017

please insert values, csv_address or csv_mls: 
value
How many mls you want to check?
1
Enter the 1st #MLS
14765328

This value exists in Website

This value exists in Website

This value exists in Website
None


# No Need to Run

In [ ]:
login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()


result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(ListingId = {})'.format(20018438))
for item in result:
  print(item['PostalCode'])

In [ ]:
df = pd.read_csv('test.csv')
df.rename(columns={ df.columns[0]: "地址" }, inplace = True)
df.to_csv('test1.csv')

In [ ]:
#df.rename(columns={ df.columns[0]: "地址" }, inplace = True)
#df.to_csv('test1.csv')